<h2>Lab1: Базове знайомство з Apache Spark</h2>

In [ ]:
import os
os.environ['SPARK_HOME'] = "/home/zaranik/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import Column

<h4> Завдання 1: Створити кілька своїх DataFrames. Виконати над ними операції
об’єднання, додавання та видалення стовпців і рядків </h4>

In [ ]:
spark = SparkSession.builder.appName("lab1").getOrCreate()
spark

1. Створення DataFrames

In [ ]:
spark.createDataFrame([('Alice', 1), ]).show()

In [ ]:
d = [{'name': 'Alice', 'age': 1}, {'name': 'Bogdan', 'age': 22}]
spark.createDataFrame(d).show()

In [ ]:
spark.createDataFrame([('Alice', 1)], ['name', 'age']).show()

In [ ]:
data1 = [
  (1, 'Alice', 25),
  (2, 'Bob', 30),
  (3, 'Charlie', 35)
]
df1 = spark.createDataFrame(data1, ["id", "name", "age"])
df1.show()

In [ ]:
data2 = [
  (4, 'David', 40),
  (5, 'Eva', 45),
  (3, 'Charlie', 35)
]
df2 = spark.createDataFrame(data2, ["id", "name", "age"])
df2.show()

2. Операція об'єднання DataFrames (Union)

In [ ]:
union_df = df1.union(df2).distinct()
union_df.show()

3. Додавання стовпця

In [ ]:
from pyspark.sql.functions import lit

df_with_new_column = union_df.withColumn("country", lit("Unknown"))
df_with_new_column.show()

4. Видалення стовпця

In [ ]:
# Видаляємо стовпець "Age"
df_without_column = df_with_new_column.drop("age")
df_without_column.show()

5. Видалення рядка

In [ ]:
# Видаляємо рядок з ID = 2
df_without_row = df_with_new_column.filter(df_with_new_column["id"] != 2)
df_without_row.show()

6. Додавання рядка

In [ ]:
new_row = spark.createDataFrame([(6, "Bogdan", 22, "Ukraine")], ["id", "name", "age", "country"])
df_with_new_row = df_with_new_column.union(new_row)
df_with_new_row.show()

In [ ]:
spark.stop()

<h4> Завдання 2: Створити свою схему та завантажити дані з файлу, виконати дії
відповідно до варіанту </h4>

0. Importing data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
spark = SparkSession.builder.appName("lab1-frogs").getOrCreate()

In [ ]:
# Define the schema for the data
frog_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("pres.abs", IntegerType(), False),
    StructField("northing", IntegerType(), False),
    StructField("easting", IntegerType(), False),
    StructField("altitude", IntegerType(), False),
    StructField("distance", IntegerType(), False),
    StructField("NoOfPools", IntegerType(), False),
    StructField("NoOfSites", IntegerType(), False),
    StructField("avrain", DoubleType(), False),
    StructField("meanmin", DoubleType(), False),
    StructField("meanmax", DoubleType(), False)
])

In [ ]:
frogs_df = spark.read.csv("./frogs.csv", header=True, schema=frog_schema)
frogs_df = frogs_df.withColumnRenamed("pres.abs", "presence")
frogs_df.show(5)

1. Знайти максимальну висоту, на якій спостерігаються жаби.

In [ ]:
max_altitude = frogs_df.filter(frogs_df["presence"] == 1).agg({"altitude": "max"})
max_altitude.show()

2. Вивести всі висоти, де середні максимальні температури не
перевищують 14 градусів і на яких спостерігались жаби.

In [ ]:
df_filtered = frogs_df.filter((frogs_df["presence"] == 1) & (frogs_df["meanmax"] <= 14))
df_filtered.select("altitude").show(10)

3. Відсортувати місця, де спостерігались жаби, за відстанню до
найближчого поселення (спадаючий порядок).

In [ ]:
frogs_df.filter(frogs_df["presence"] == 1)
df_sorted = df_filtered.orderBy(frogs_df["distance"].desc())
df_sorted.show()

4. Порахувати загальну кількість груп для розмноження.

In [ ]:
from pyspark.sql.functions import sum
total_pools = frogs_df.agg(sum("NoOfPools"))
print("Загальна кількість груп для розмноження:\n")
total_pools.show()

In [ ]:
spark.stop()